In [1]:
import os
import yaml
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from composio_langchain import ComposioToolSet, App
from langchain.memory import ConversationBufferMemory
from langchain.tools import StructuredTool
from langchain_core.tools import ToolException
import warnings

warnings.filterwarnings("ignore")

/Users/morossini/anaconda3/envs/composio/lib/python3.12/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/morossini/anaconda3/envs/composio/lib/python3.12/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
class MyAgent:
    def __init__(self, tools, custom_prompt: str = None):
        self.llm = ChatOpenAI(
            model="gpt-4o",
            api_key=os.getenv("OPENAI_API_KEY"),
        )
        self.tools = tools
        self.agent_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.agent = create_tool_calling_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=ChatPromptTemplate.from_messages([
                ("system", custom_prompt if custom_prompt else "You are an experienced assistant. You are able to perform any task asked by the user through all the tools you have access to. Your goal is to complete the user's task by using the tools you have access to."),
                ("placeholder", "{chat_history}"),
                ("human", "{input}"),
                ("placeholder", "{agent_scratchpad}"),
            ]),
        )
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, memory=self.agent_memory, verbose=True)

    def myinvoke(self, input: str):
        result = self.executor.invoke({"input": input})
        print(result)
        print("\n-----------------------------------")
        print("Agent Response:")
        print(result['output'])
        print("-----------------------------------")

In [3]:
composio_toolset = ComposioToolSet(api_key=os.getenv('COMPOSIO_API_KEY'))
agent_tools = composio_toolset.get_tools(apps=[App.CLICKUP])

In [4]:
def try_except_tool(func):
    try:
        return func
    except ToolException as e:
        print(f"Error: {e}")

In [5]:
enhanced_agent_tools = [
    StructuredTool(
        name=tool.name,
        description=tool.description,
        args_schema=tool.args_schema,
        func=try_except_tool(tool.func),
        handle_tool_error=True,
        handle_validation_error=True
    ) for tool in agent_tools
]

In [6]:
agent = MyAgent(tools=enhanced_agent_tools)

In [7]:
agent.myinvoke("Retrieve the name of my clickup workspace")



> Entering new AgentExecutor chain...


python(86269) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



Invoking: `clickup_authorization_view_account_details` with `{}`


{'execution_details': {'executed': True}, 'response_data': {'user': {'id': 81918955, 'username': 'João Guilherme Silva Morossini', 'email': 'joaog.morossini@gmail.com', 'color': '#81b1ff', 'profilePicture': 'https://attachments.clickup.com/profilePictures/81918955_kcK.jpg', 'initials': 'JM', 'week_start_day': 1, 'global_font_support': True, 'timezone': 'America/Sao_Paulo'}}}
Invoking: `clickup_authorization_get_work_space_list` with `{}`


{'execution_details': {'executed': True}, 'response_data': {'teams': [{'id': '12927880', 'name': 'Cogmo Workspace', 'color': '#202020', 'avatar': 'https://attachments2.clickup.com/team_avatars/12927880_9xT.blob?Expires=1735603200&Key-Pair-Id=APKAIYJRUKB5RLQWHZWQ&Signature=IjQQWt2qYTYv5yXQn3jhIfm5i9-ETIQdodisZqlZDGzoY4qAE5qvv~0FHDxbzSqeN0Rd-osZw5sJIpIpYWO1SPzvm2u-gVJgy9RGaaB4KtnMyZw-wStnG67hevdPdOukxHFE~VLEEP2HoHSnYyWyjUrMFmA0f256ZsU8hhM4Lq8JuUfFJyi69HbCD2l~F8kCeLOadW4z0ofkTTMfVNjDrqe

In [8]:
execution_details_1 = {'execution_details': {'executed': True}, 'response_data': {'teams': [{'id': '12927880', 'name': 'Cogmo Workspace', 'color': '#202020', 'avatar': 'https://attachments2.clickup.com/team_avatars/12927880_9xT.blob?Expires=1735603200&Key-Pair-Id=APKAIYJRUKB5RLQWHZWQ&Signature=IjQQWt2qYTYv5yXQn3jhIfm5i9-ETIQdodisZqlZDGzoY4qAE5qvv~0FHDxbzSqeN0Rd-osZw5sJIpIpYWO1SPzvm2u-gVJgy9RGaaB4KtnMyZw-wStnG67hevdPdOukxHFE~VLEEP2HoHSnYyWyjUrMFmA0f256ZsU8hhM4Lq8JuUfFJyi69HbCD2l~F8kCeLOadW4z0ofkTTMfVNjDrqezFkiL72Rlr3v~bmq3oCGKIaAMZFWyorq3ru5HDJvphqudIkrFpA56kIgESPIzzU3lkINk7CVHJ2xKRRnU4ruo4cjufxo5RAtL9RehP5gaP8r2A1BmgDlZ5uLTUiK0kg__', 'members': [{'user': {'id': 44309342, 'username': 'Hibrael Loures', 'email': 'ibraelmoura@gmail.com', 'color': None, 'profilePicture': 'https://attachments.clickup.com/profilePictures/44309342_YHC.jpg', 'initials': 'HL', 'role': 3, 'custom_role': None, 'last_active': '1719956684066', 'date_joined': '1719500887900', 'date_invited': '1719500735732'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}}, {'user': {'id': 82061927, 'username': 'Rafael Alves Magalhães', 'email': 'magalhaesrafael07@gmail.com', 'color': '', 'profilePicture': None, 'initials': 'RM', 'role': 3, 'custom_role': None, 'last_active': '1722013249094', 'date_joined': '1715710208104', 'date_invited': '1715710148238'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}}, {'user': {'id': 3115657, 'username': 'Filipe Werlang', 'email': 'filipewerlang.d@gmail.com', 'color': '#622aea', 'profilePicture': 'https://attachments.clickup.com/profilePictures/3115657_GPh.jpg', 'initials': 'FW', 'role': 3, 'custom_role': None, 'last_active': '1716570383387', 'date_joined': '1713903403635', 'date_invited': '1713903340285'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}}, {'user': {'id': 72816031, 'username': 'Gabriel Silvestre', 'email': 'gabriel.h.silvestre11@gmail.com', 'color': '', 'profilePicture': None, 'initials': 'GS', 'role': 4, 'custom_role': None, 'last_active': '1711651350955', 'date_joined': '1706705323185', 'date_invited': '1706703387889'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 81955248, 'username': 'Sara Lima dos Santos', 'email': 'zarahlimma@gmail.com', 'color': '#02579b', 'profilePicture': None, 'initials': 'SS', 'role': 4, 'custom_role': None, 'last_active': '1705520398448', 'date_joined': '1700762889153', 'date_invited': '1700762748501'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 81942227, 'username': None, 'email': 'asdf@hbnjg.com', 'color': None, 'profilePicture': None, 'initials': 'A', 'role': 4, 'custom_role': None, 'last_active': None, 'date_joined': None, 'date_invited': '1697648867625'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 81942111, 'username': 'Romulo Fernando', 'email': 'fssromulo@gmail.com', 'color': '#ff897f', 'profilePicture': None, 'initials': 'RF', 'role': 4, 'custom_role': None, 'last_active': '1697657536419', 'date_joined': '1697639835143', 'date_invited': '1697639162077'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 32424453, 'username': 'Marcelo Marra', 'email': 'marcelo@devsquad.email', 'color': '#ffa12f', 'profilePicture': None, 'initials': 'MM', 'role': 4, 'custom_role': None, 'last_active': '1697856401732', 'date_joined': '1697578574655', 'date_invited': '1697578470743'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 81941856, 'username': 'Joseph Torres Kaltenecker', 'email': 'joseph.kaltenecker@gmail.com', 'color': '#d60800', 'profilePicture': None, 'initials': 'JK', 'role': 4, 'custom_role': None, 'last_active': '1697684220484', 'date_joined': '1697584232755', 'date_invited': '1697569495566'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 42954980, 'username': 'Paulo Gontijo', 'email': 'paulo@devsquad.email', 'color': '#cddc39', 'profilePicture': None, 'initials': 'PG', 'role': 4, 'custom_role': None, 'last_active': '1697629728042', 'date_joined': '1697564620641', 'date_invited': '1697549149854'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 81934261, 'username': 'Davi Mello de Figueiredo', 'email': 'davi.figueiredo@devsquad.email', 'color': '#02579b', 'profilePicture': None, 'initials': 'DF', 'role': 4, 'custom_role': None, 'last_active': '1697515706191', 'date_joined': '1696005734675', 'date_invited': '1695853301124'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 32443176, 'username': 'Wellington Mota', 'email': 'wellington@devsquad.email', 'color': '#006063', 'profilePicture': None, 'initials': 'WM', 'role': 4, 'custom_role': None, 'last_active': '1697218599283', 'date_joined': '1696015880511', 'date_invited': '1695853300919'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 54949151, 'username': 'DAC CAPITAL', 'email': 'gerencial@daccapital.com.br', 'color': '#2ea52c', 'profilePicture': None, 'initials': 'DC', 'role': 4, 'custom_role': None, 'last_active': '1702326584933', 'date_joined': '1702326323812', 'date_invited': '1695150324637'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 81930584, 'username': None, 'email': 'raquel.cardoso@daccapital.br', 'color': None, 'profilePicture': None, 'initials': 'R', 'role': 4, 'custom_role': None, 'last_active': None, 'date_joined': None, 'date_invited': '1695150306102'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 54951651, 'username': 'Legal', 'email': 'legal@daccapital.com.br', 'color': '#622aea', 'profilePicture': None, 'initials': 'L', 'role': 4, 'custom_role': None, 'last_active': '1695665069022', 'date_joined': '1695151113312', 'date_invited': '1694655863772'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 81918955, 'username': 'João Guilherme Silva Morossini', 'email': 'joaog.morossini@gmail.com', 'color': '#81b1ff', 'profilePicture': 'https://attachments.clickup.com/profilePictures/81918955_kcK.jpg', 'initials': 'JM', 'role': 3, 'custom_role': None, 'last_active': '1722013216886', 'date_joined': '1693245712205', 'date_invited': '1693245455315'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}}, {'user': {'id': 55040420, 'username': 'Gabriela Barreto Alberton', 'email': 'gabibalberton@gmail.com', 'color': '#f8306d', 'profilePicture': 'https://attachments.clickup.com/profilePictures/55040420_3Pe.jpg', 'initials': 'GA', 'role': 3, 'custom_role': None, 'last_active': '1695936684276', 'date_joined': '1671653244700', 'date_invited': '1671653134716'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}}, {'user': {'id': 3195751, 'username': 'Nicolas Augusto França Melo', 'email': 'nicolasfmelo@gmail.com', 'color': '#ff4081', 'profilePicture': 'https://attachments.clickup.com/profilePictures/3195751_KsS.jpg', 'initials': 'NM', 'role': 4, 'custom_role': None, 'last_active': '1701520726948', 'date_joined': '1670092173360', 'date_invited': '1670092062140'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 54979221, 'username': 'Caio Ferreira da Costa Macedo', 'email': 'cmacedo2011@hotmail.com', 'color': '#ff4081', 'profilePicture': None, 'initials': 'CM', 'role': 3, 'custom_role': None, 'last_active': '1720449195119', 'date_joined': '1667955977241', 'date_invited': '1667952178032'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}}, {'user': {'id': 54969508, 'username': 'Jessé Dantas Macedo Santos de Jesus', 'email': 'jessedantasti@gmail.com', 'color': '#388d3c', 'profilePicture': None, 'initials': 'JJ', 'role': 4, 'custom_role': None, 'last_active': '1719953610878', 'date_joined': '1667507094940', 'date_invited': '1667506942142'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 54951654, 'username': 'RAQUEL AMARAL CARDOSO', 'email': 'raquel.cardoso@daccapital.com.br', 'color': '#622aea', 'profilePicture': None, 'initials': 'RC', 'role': 4, 'custom_role': None, 'last_active': '1675922064014', 'date_joined': '1667487082668', 'date_invited': '1667356810385'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 54939390, 'username': 'gisleine', 'email': 'gbalberton@gmail.com', 'color': None, 'profilePicture': None, 'initials': 'G', 'role': 3, 'custom_role': None, 'last_active': '1709398469540', 'date_joined': '1666206215365', 'date_invited': '1666206167107'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}}, {'user': {'id': 49198435, 'username': 'Rafael Macedo Mendonça', 'email': 'rafael.macedo@perceptus.ai', 'color': '#5d4037', 'profilePicture': None, 'initials': 'RM', 'role': 3, 'custom_role': None, 'last_active': '1721686409328', 'date_joined': '1664404432181', 'date_invited': '1664404270203'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}}, {'user': {'id': 49198378, 'username': 'Ana Carolina Dos Santos', 'email': 'ana.carolina0509@gmail.com', 'color': '#795548', 'profilePicture': None, 'initials': 'AS', 'role': 4, 'custom_role': None, 'last_active': '1708656179438', 'date_joined': '1664410489464', 'date_invited': '1664401974533'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 49193170, 'username': 'Ricardo Terra', 'email': 'rterra88@gmail.com', 'color': '#07a092', 'profilePicture': None, 'initials': 'RT', 'role': 4, 'custom_role': None, 'last_active': '1720470355536', 'date_joined': '1664219470550', 'date_invited': '1664219444037'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 3027211, 'username': 'Denisson test', 'email': 'denissonfernades@gmail.com', 'color': '#afb42b', 'profilePicture': None, 'initials': 'DT', 'role': 4, 'custom_role': None, 'last_active': '1667362259881', 'date_joined': '1656636833254', 'date_invited': '1656633124280'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}, 'can_see_time_spent': True, 'can_see_time_estimated': True, 'can_see_points_estimated': True, 'can_edit_tags': False, 'can_create_views': False}, {'user': {'id': 48772077, 'username': 'Hélio Potelicki', 'email': 'helio@cogmo.com.br', 'color': '#2ea52c', 'profilePicture': 'https://attachments.clickup.com/profilePictures/48772077_rvb.jpg', 'initials': 'HP', 'role': 3, 'custom_role': None, 'last_active': '1721862149787', 'date_joined': '1649266868990', 'date_invited': '1649266363940'}, 'invited_by': {'id': 18951490, 'username': 'Gabriel Alberton', 'color': '#7b68ee', 'email': 'gabriel@cogmo.com.br', 'initials': 'GA', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg'}}, {'user': {'id': 18951490, 'username': 'Gabriel Alberton', 'email': 'gabriel@cogmo.com.br', 'color': '#7b68ee', 'profilePicture': 'https://attachments.clickup.com/profilePictures/18951490_W33.jpg', 'initials': 'GA', 'role': 1, 'custom_role': None, 'last_active': '1721995000095', 'date_joined': '1632348127957', 'date_invited': '1632348127957'}}]}]}}

In [9]:
from pprint import pprint

pprint(execution_details_1)

{'execution_details': {'executed': True},
 'response_data': {'teams': [{'avatar': 'https://attachments2.clickup.com/team_avatars/12927880_9xT.blob?Expires=1735603200&Key-Pair-Id=APKAIYJRUKB5RLQWHZWQ&Signature=IjQQWt2qYTYv5yXQn3jhIfm5i9-ETIQdodisZqlZDGzoY4qAE5qvv~0FHDxbzSqeN0Rd-osZw5sJIpIpYWO1SPzvm2u-gVJgy9RGaaB4KtnMyZw-wStnG67hevdPdOukxHFE~VLEEP2HoHSnYyWyjUrMFmA0f256ZsU8hhM4Lq8JuUfFJyi69HbCD2l~F8kCeLOadW4z0ofkTTMfVNjDrqezFkiL72Rlr3v~bmq3oCGKIaAMZFWyorq3ru5HDJvphqudIkrFpA56kIgESPIzzU3lkINk7CVHJ2xKRRnU4ruo4cjufxo5RAtL9RehP5gaP8r2A1BmgDlZ5uLTUiK0kg__',
                              'color': '#202020',
                              'id': '12927880',
                              'members': [{'invited_by': {'color': '#7b68ee',
                                                          'email': 'gabriel@cogmo.com.br',
                                                          'id': 18951490,
                                                          'initials': 'GA',
                           

In [10]:
execution_details_1.keys()

dict_keys(['execution_details', 'response_data'])

In [13]:
execution_details_1['response_data'].keys()

dict_keys(['teams'])

In [15]:
len(execution_details_1['response_data']['teams'])

1

In [16]:
execution_details_1['response_data']['teams'][0].keys()

dict_keys(['id', 'name', 'color', 'avatar', 'members'])

In [18]:
print(execution_details_1['response_data']['teams'][0]['id'])
type(execution_details_1['response_data']['teams'][0]['id'])

12927880


str

In [19]:
clickup_team_id="12927880"

In [21]:
execution_details_1['response_data']['teams'][0]

{'id': '12927880',
 'name': 'Cogmo Workspace',
 'color': '#202020',
 'avatar': 'https://attachments2.clickup.com/team_avatars/12927880_9xT.blob?Expires=1735603200&Key-Pair-Id=APKAIYJRUKB5RLQWHZWQ&Signature=IjQQWt2qYTYv5yXQn3jhIfm5i9-ETIQdodisZqlZDGzoY4qAE5qvv~0FHDxbzSqeN0Rd-osZw5sJIpIpYWO1SPzvm2u-gVJgy9RGaaB4KtnMyZw-wStnG67hevdPdOukxHFE~VLEEP2HoHSnYyWyjUrMFmA0f256ZsU8hhM4Lq8JuUfFJyi69HbCD2l~F8kCeLOadW4z0ofkTTMfVNjDrqezFkiL72Rlr3v~bmq3oCGKIaAMZFWyorq3ru5HDJvphqudIkrFpA56kIgESPIzzU3lkINk7CVHJ2xKRRnU4ruo4cjufxo5RAtL9RehP5gaP8r2A1BmgDlZ5uLTUiK0kg__',
 'members': [{'user': {'id': 44309342,
    'username': 'Hibrael Loures',
    'email': 'ibraelmoura@gmail.com',
    'color': None,
    'profilePicture': 'https://attachments.clickup.com/profilePictures/44309342_YHC.jpg',
    'initials': 'HL',
    'role': 3,
    'custom_role': None,
    'last_active': '1719956684066',
    'date_joined': '1719500887900',
    'date_invited': '1719500735732'},
   'invited_by': {'id': 18951490,
    'username': 'Gabr

In [24]:
clickup_team = execution_details_1['response_data']['teams'][0]

In [26]:
len(clickup_team['members'])

28

In [27]:
agent.myinvoke(f"Which tools do you have for working with Clickup spaces?")



> Entering new AgentExecutor chain...
Here are the tools available for working with ClickUp Spaces:

1. **View the Spaces available in a Workspace**:
   - Function: `clickup_spaces_get_space_details`
   - Parameters: `{ team_id: number, archived?: boolean }`

2. **Add a new Space to a Workspace**:
   - Function: `clickup_spaces_add_new_space_to_work_space`
   - Parameters: `{ team_id: number, name: string, multiple_assignees: boolean, features__tags__enabled: boolean, features__due_dates__enabled: boolean, features__due_dates__start_date: boolean, features__due_dates__remap_due_dates: boolean, features__due_dates__remap_closed_due_date: boolean, features__time_tracking__enabled: boolean, features__time_estimates__enabled: boolean, features__checklists__enabled: boolean, features__custom_fields__enabled: boolean, features__remap_dependencies__enabled: boolean, features__dependency_warning__enabled: boolean, features__portfolios__enabled: boolean }`

3. **View the Spaces available in a

In [28]:
agent.myinvoke(f"Use 'clickup_spaces_get_space_details' with the following team_id to view available Spaces in my Workspace. team_id: {clickup_team_id}")



> Entering new AgentExecutor chain...

Invoking: `clickup_spaces_get_space_details` with `{'team_id': 12927880}`


{'execution_details': {'executed': False}, 'response_data': {'error': '{"err":"invalid input syntax for integer: \\"12927880.0\\"","ECODE":"SHARD_001"}', 'status_code': 500}}
Invoking: `clickup_authorization_get_work_space_list` with `{}`
responded: It appears there was an error with the request due to an invalid input syntax. The `team_id` seems to be interpreted incorrectly.

Let's try the request again using the corrected `team_id`.

{'execution_details': {'executed': True}, 'response_data': {'teams': [{'id': '12927880', 'name': 'Cogmo Workspace', 'color': '#202020', 'avatar': 'https://attachments2.clickup.com/team_avatars/12927880_9xT.blob?Expires=1735603200&Key-Pair-Id=APKAIYJRUKB5RLQWHZWQ&Signature=IjQQWt2qYTYv5yXQn3jhIfm5i9-ETIQdodisZqlZDGzoY4qAE5qvv~0FHDxbzSqeN0Rd-osZw5sJIpIpYWO1SPzvm2u-gVJgy9RGaaB4KtnMyZw-wStnG67hevdPdOukxHFE~VLEEP2HoHSnYyWyjUrMFmA0f256ZsU8hhM4L

In [31]:
agent.myinvoke("Use 'clickup_spaces_get_space_details' with the following team_id to view available Spaces in my Workspace. team_id:'12927880'. You MUST pass the team_id as an integer, NOT as a float or as a string")



> Entering new AgentExecutor chain...

Invoking: `clickup_spaces_get_space_details` with `{'team_id': 12927880}`


{'execution_details': {'executed': False}, 'response_data': {'error': '{"err":"invalid input syntax for integer: \\"12927880.0\\"","ECODE":"SHARD_001"}', 'status_code': 500}}
Invoking: `clickup_authorization_get_work_space_list` with `{}`
responded: It seems there was an issue with the request. The API returned an error indicating an invalid input syntax for the `team_id`.

I'll ensure the `team_id` is properly formatted as an integer and try again.

{'execution_details': {'executed': True}, 'response_data': {'teams': [{'id': '12927880', 'name': 'Cogmo Workspace', 'color': '#202020', 'avatar': 'https://attachments2.clickup.com/team_avatars/12927880_9xT.blob?Expires=1735603200&Key-Pair-Id=APKAIYJRUKB5RLQWHZWQ&Signature=IjQQWt2qYTYv5yXQn3jhIfm5i9-ETIQdodisZqlZDGzoY4qAE5qvv~0FHDxbzSqeN0Rd-osZw5sJIpIpYWO1SPzvm2u-gVJgy9RGaaB4KtnMyZw-wStnG67hevdPdOukxHFE~VLEEP2HoHSnYyWyjUrMFmA

In [35]:
type(enhanced_agent_tools)

list

In [82]:
enhanced_agent_tools

[StructuredTool(name='clickup_attachments_upload_file_to_task_as_attachment', description='Upload a file to a task as an attachment.  ***Note:** This request uses multipart/form-data as the content type.*', args_schema=<class 'pydantic.v1.main.attachments_upload_file_to_task_as_attachmentRequest'>, handle_tool_error=True, handle_validation_error=True, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x1256f7a60>),
 StructuredTool(name='clickup_authorization_get_access_token', description="These are the routes for authing the API and going through the [OAuth flow](https://clickup.com/api). Applications utilizing a personal API token don't use this endpoint. ***Note:** OAuth tokens are not supported when using the [**Try It** feature](https://clickup.com/api) of our Reference docs. You can't try this endpoint from your web browser.*", args_schema=<class 'pydantic.v1.main.authorization_get_access_tokenRequest'>, handle_tool_error=True, handle_validation_error=True, func=<f

In [81]:
for i in range(len(enhanced_agent_tools)):
    if "clickup_spaces" in enhanced_agent_tools[i].name:
        print(f"{i}) {enhanced_agent_tools[i].name}: {enhanced_agent_tools[i].description}")

60) clickup_spaces_get_space_details: View the Spaces avialable in a Workspace. You can only get member info in private Spaces.
61) clickup_spaces_add_new_space_to_work_space: Add a new Space to a Workspace.
62) clickup_spaces_get_details: View the Spaces available in a Workspace.
63) clickup_spaces_update_details_and_enable_click_apps: Rename, set the Space color, and enable ClickApps for a Space.
64) clickup_spaces_remove_space: Delete a Space from your Workspace.


In [40]:
enhanced_agent_tools[60]

StructuredTool(name='clickup_spaces_get_space_details', description='View the Spaces avialable in a Workspace. You can only get member info in private Spaces.', args_schema=<class 'pydantic.v1.main.spaces_get_space_detailsRequest'>, handle_tool_error=True, handle_validation_error=True, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x125a17f60>)

In [41]:
enhanced_agent_tools[62]

StructuredTool(name='clickup_spaces_get_details', description='View the Spaces available in a Workspace.', args_schema=<class 'pydantic.v1.main.spaces_get_detailsRequest'>, handle_tool_error=True, handle_validation_error=True, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x125b94b80>)

In [9]:
spaces_details = enhanced_agent_tools[62]

In [43]:
dir(spaces_details)

['Config',
 'InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_updat

In [50]:
print(spaces_details.json)

<bound method BaseModel.json of StructuredTool(name='clickup_spaces_get_details', description='View the Spaces available in a Workspace.', args_schema=<class 'pydantic.v1.main.spaces_get_detailsRequest'>, handle_tool_error=True, handle_validation_error=True, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x125b94b80>)>


In [54]:
spaces_details.invoke({"team_id": "12927880"})

'Tool input validation error'

In [10]:
spaces_details.invoke({"team_id": 12927880})

'Tool input validation error'

In [57]:
spaces_details.get_input_schema

<bound method BaseTool.get_input_schema of StructuredTool(name='clickup_spaces_get_details', description='View the Spaces available in a Workspace.', args_schema=<class 'pydantic.v1.main.spaces_get_detailsRequest'>, handle_tool_error=True, handle_validation_error=True, func=<function ComposioToolSet._wrap_action.<locals>.function at 0x125b94b80>)>

In [60]:
dir(spaces_details.args_schema)

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '_abc_impl',
 '_calculate_keys',
 '_copy_and_set_values',
 '_decompose_class',
 '_enfo

In [62]:
spaces_details.args_schema.schema()

{'title': 'spaces_get_detailsRequest',
 'type': 'object',
 'properties': {'space_id': {'title': 'Space Id',
   'examples': [],
   'type': 'number'}},
 'required': ['space_id']}

In [79]:
spaces_details.invoke({"space_id": "54804921"})

{'execution_details': {'executed': False},
 'response_data': {'error': '{"err":"Space ID invalid","ECODE":"INPUT_002"}',
  'status_code': 400}}

In [66]:
agent.myinvoke("What can you tell me about my Clickup space of 54804921?")



> Entering new AgentExecutor chain...

Invoking: `clickup_spaces_get_space_details` with `{'space_id': 54804921}`


Tool input validation error
Invoking: `clickup_spaces_get_details` with `{'space_id': 54804921}`
responded: It appears there was an input validation error while trying to retrieve the details for the space with `space_id: 54804921`. Let's try again to ensure the request is formatted correctly.

{'execution_details': {'executed': False}, 'response_data': {'error': '{"err":"Space ID invalid","ECODE":"INPUT_002"}', 'status_code': 400}}It seems that the `space_id` provided, `54804921`, is invalid. Please verify the `space_id` and try again. If you have any other valid space IDs or different requests, please let me know!

> Finished chain.
{'input': 'What can you tell me about my Clickup space of 54804921?', 'chat_history': [HumanMessage(content='Retrieve the name of my clickup workspace'), AIMessage(content='The name of your ClickUp workspace is **"Cogmo Workspace"**.'), Hu

In [68]:
agent.myinvoke("Can you give me a list of Spaces in my clickup workspace?")



> Entering new AgentExecutor chain...

Invoking: `clickup_spaces_get_space_details` with `{'team_id': 12927880}`


{'execution_details': {'executed': False}, 'response_data': {'error': '{"err":"invalid input syntax for integer: \\"12927880.0\\"","ECODE":"SHARD_001"}', 'status_code': 500}}
Invoking: `clickup_authorization_get_work_space_list` with `{}`
responded: It looks like there was a problem with interpreting the `team_id`. I will attempt to retrieve the spaces again, ensuring that the `team_id` is passed correctly as an integer.

Let's try this again.

{'execution_details': {'executed': True}, 'response_data': {'teams': [{'id': '12927880', 'name': 'Cogmo Workspace', 'color': '#202020', 'avatar': 'https://attachments2.clickup.com/team_avatars/12927880_9xT.blob?Expires=1735603200&Key-Pair-Id=APKAIYJRUKB5RLQWHZWQ&Signature=IjQQWt2qYTYv5yXQn3jhIfm5i9-ETIQdodisZqlZDGzoY4qAE5qvv~0FHDxbzSqeN0Rd-osZw5sJIpIpYWO1SPzvm2u-gVJgy9RGaaB4KtnMyZw-wStnG67hevdPdOukxHFE~VLEEP2HoHSnYyWyjUrMFmA0f256Z

In [ ]:
spaces_details

In [ ]:
https://app.clickup.com/12927880/v/s/54804921

In [ ]:
agent.myinvoke("What did I ask before?")

In [ ]:
agent.myinvoke("Which tools do you have access to?")

In [ ]:
agent.myinvoke("Can you retrieve a list of spaces from my workspace?")

In [ ]:
agent.myinvoke("Retrieve a list of members in my workspace")

In [84]:
clickup_team_id

12927880

In [87]:
tool_schema = """
{'title': 'spaces_get_detailsRequest',
 'type': 'object',
 'properties': {'space_id': {'title': 'Space Id',
   'examples': [],
   'type': 'number'}},
 'required': ['space_id']}
 """

clickup_team_id = 12927880

clickup_space_id = 54804921

In [88]:
input = f"""Use the information below to retrieve information about a Clickup space
- ClickUp Team ID: {clickup_team_id}
- ClickUp Space ID: {clickup_space_id}
- Tool: clickup_spaces_get_space_details\
- Tool Schema:
{tool_schema}
"""

print(input)

Use the information below to retrieve information about a Clickup space
- ClickUp Team ID: 12927880
- ClickUp Space ID: 54804921
- Tool: clickup_spaces_get_space_details- Tool Schema:

{'title': 'spaces_get_detailsRequest',
 'type': 'object',
 'properties': {'space_id': {'title': 'Space Id',
   'examples': [],
   'type': 'number'}},
 'required': ['space_id']}
 





In [89]:
agent.myinvoke("input")



> Entering new AgentExecutor chain...

Invoking: `clickup_authorization_get_work_space_list` with `{}`


{'execution_details': {'executed': True}, 'response_data': {'teams': [{'id': '12927880', 'name': 'Cogmo Workspace', 'color': '#202020', 'avatar': 'https://attachments2.clickup.com/team_avatars/12927880_9xT.blob?Expires=1735603200&Key-Pair-Id=APKAIYJRUKB5RLQWHZWQ&Signature=IjQQWt2qYTYv5yXQn3jhIfm5i9-ETIQdodisZqlZDGzoY4qAE5qvv~0FHDxbzSqeN0Rd-osZw5sJIpIpYWO1SPzvm2u-gVJgy9RGaaB4KtnMyZw-wStnG67hevdPdOukxHFE~VLEEP2HoHSnYyWyjUrMFmA0f256ZsU8hhM4Lq8JuUfFJyi69HbCD2l~F8kCeLOadW4z0ofkTTMfVNjDrqezFkiL72Rlr3v~bmq3oCGKIaAMZFWyorq3ru5HDJvphqudIkrFpA56kIgESPIzzU3lkINk7CVHJ2xKRRnU4ruo4cjufxo5RAtL9RehP5gaP8r2A1BmgDlZ5uLTUiK0kg__', 'members': [{'user': {'id': 44309342, 'username': 'Hibrael Loures', 'email': 'ibraelmoura@gmail.com', 'color': None, 'profilePicture': 'https://attachments.clickup.com/profilePictures/44309342_YHC.jpg', 'initials': 'HL', 'role': 3, 'custom_role': None, 'last_active': '1719